# The Cancer Gene Atlas Cancer (TCGA) - Lung Adenocarcinoma (LUAD)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Mon Apr 30 16:26:21 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Lung adenocarcinoma [LUAD]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-49-4507,3744,LUAD,TCGA,50b2c647-aecd-4c81-af22-0d8116593552,49,73.0,NaN,58,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,1991.0,NaN,NaN
TCGA-64-5774,3745,LUAD,TCGA,fd3fc273-c035-40ea-a459-ff4806f5d592,64,60.0,NaN,84,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,2004.0,NaN,1959.0
TCGA-05-4424,3747,LUAD,TCGA,66763a0c-6cda-4832-a0cc-e7b496d78eaa,05,70.0,R-Upper,144,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,2007.0,NaN,1957.0
TCGA-05-4417,3748,LUAD,TCGA,a244a99a-d7cc-4fb1-bc76-b66886481621,05,51.0,R-Upper,144,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,2008.0,NaN,1971.0
TCGA-05-4382,3749,LUAD,TCGA,3434b91a-c05f-460f-a078-7b1bb6e7085d,05,68.0,L-Upper,58,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,"Yes, History of Prior Malignancy",Penis,"Other, specify",Prior Malignancy,NaN,NaN,NaN


In [16]:
sample_meta.shape

(522, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_LUAD_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,c8eda0ee-e5e3-4847-8c9d-c816811d6e41,4c7c50d7-4114-41e4-a154-388b5a2ec785,f28adde4-2132-498a-8711-4364ac750c42,8a70b5bd-693b-46f4-b04c-a8365ad609ba,44704283-60fd-45f5-9ad0-a8a54901d74a,b2ac6eaf-b44b-47f9-824d-9bc3126d9b0a,8d40f07a-dfb9-484d-a49f-3c215e793cc7,57cf971c-0cea-40a4-a475-183ad5e3f016,8cb8eaa0-2998-44d9-8324-8e2347287ed4,b7ac921c-0808-47a8-bf1f-65f4800e3863,...,0650ea78-95e5-4a5b-a349-fbe6b85b6ab3,43536117-bc15-4459-a7ef-628757985f09,b50427f3-588c-464c-836b-c00cc9b1be34,e5706901-9ebb-4960-8558-5f28b4554cd3,d76f678e-79d8-4e7f-989a-c6d75004808a,48b01bcd-68ed-4a2f-847a-0a9501bed213,5502d56e-a135-4c91-aa43-0cc71f508f91,f64f899b-10db-4d42-a6e5-ad42299c92eb,3d91f464-151b-4f29-b7bd-1520bf72297b,1656ac5d-25d7-4027-a01b-c9d0c82856f3
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,92.642800,231.355000,197.86700,166.32700,239.04200,205.900000,105.2290,150.025000,152.680000,131.470000,...,134.466000,183.4860,90.798100,87.662600,230.199000,87.5706,110.064000,155.17500,81.53430,100.87900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,13.416100,37.246600,26.86280,32.23940,24.32130,27.363300,30.3292,30.646300,39.905200,34.483200,...,10.312700,0.0000,30.594200,32.774600,0.449073,3.1280,29.277800,42.09270,34.22870,44.67800
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,8.231030,15.566200,26.54580,27.97360,31.72550,30.049300,34.5154,29.175500,19.035300,21.015600,...,25.002400,15.5547,15.766400,20.753600,10.761600,10.5089,21.214300,30.90880,8.79188,13.20930
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,19.609300,18.158200,27.79620,86.77310,43.17470,63.108700,46.4904,20.757300,48.915800,58.970800,...,16.641200,27.7776,40.457400,43.506900,24.455700,50.7069,30.995300,55.96580,20.16570,18.87750
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.486023,0.497573,1.13765,1.12786,0.07823,0.114736,0.5554,0.260879,0.419212,0.706074,...,0.054276,0.0000,0.332898,0.835527,0.000000,0.0000,0.158432,0.56765,1.98550,5.37379


In [19]:
matrix.shape

(199169, 601)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,c8eda0ee-e5e3-4847-8c9d-c816811d6e41,4c7c50d7-4114-41e4-a154-388b5a2ec785,f28adde4-2132-498a-8711-4364ac750c42,8a70b5bd-693b-46f4-b04c-a8365ad609ba,44704283-60fd-45f5-9ad0-a8a54901d74a,b2ac6eaf-b44b-47f9-824d-9bc3126d9b0a,8d40f07a-dfb9-484d-a49f-3c215e793cc7,57cf971c-0cea-40a4-a475-183ad5e3f016,8cb8eaa0-2998-44d9-8324-8e2347287ed4,b7ac921c-0808-47a8-bf1f-65f4800e3863,...,0650ea78-95e5-4a5b-a349-fbe6b85b6ab3,43536117-bc15-4459-a7ef-628757985f09,b50427f3-588c-464c-836b-c00cc9b1be34,e5706901-9ebb-4960-8558-5f28b4554cd3,d76f678e-79d8-4e7f-989a-c6d75004808a,48b01bcd-68ed-4a2f-847a-0a9501bed213,5502d56e-a135-4c91-aa43-0cc71f508f91,f64f899b-10db-4d42-a6e5-ad42299c92eb,3d91f464-151b-4f29-b7bd-1520bf72297b,1656ac5d-25d7-4027-a01b-c9d0c82856f3
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,92.642800,231.355000,197.86700,166.32700,239.04200,205.900000,105.2290,150.025000,152.680000,131.470000,...,134.466000,183.4860,90.798100,87.662600,230.199000,87.5706,110.064000,155.17500,81.53430,100.87900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,13.416100,37.246600,26.86280,32.23940,24.32130,27.363300,30.3292,30.646300,39.905200,34.483200,...,10.312700,0.0000,30.594200,32.774600,0.449073,3.1280,29.277800,42.09270,34.22870,44.67800
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,8.231030,15.566200,26.54580,27.97360,31.72550,30.049300,34.5154,29.175500,19.035300,21.015600,...,25.002400,15.5547,15.766400,20.753600,10.761600,10.5089,21.214300,30.90880,8.79188,13.20930
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,19.609300,18.158200,27.79620,86.77310,43.17470,63.108700,46.4904,20.757300,48.915800,58.970800,...,16.641200,27.7776,40.457400,43.506900,24.455700,50.7069,30.995300,55.96580,20.16570,18.87750
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.486023,0.497573,1.13765,1.12786,0.07823,0.114736,0.5554,0.260879,0.419212,0.706074,...,0.054276,0.0000,0.332898,0.835527,0.000000,0.0000,0.158432,0.56765,1.98550,5.37379


In [22]:
matrix.shape

(79930, 601)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,92.642800,231.355000,197.86700,166.32700,239.04200,205.900000,105.2290,150.025000,152.680000,131.470000,...,134.466000,183.4860,90.798100,87.662600,230.199000,87.5706,110.064000,155.17500,81.53430,100.87900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,13.416100,37.246600,26.86280,32.23940,24.32130,27.363300,30.3292,30.646300,39.905200,34.483200,...,10.312700,0.0000,30.594200,32.774600,0.449073,3.1280,29.277800,42.09270,34.22870,44.67800
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,8.231030,15.566200,26.54580,27.97360,31.72550,30.049300,34.5154,29.175500,19.035300,21.015600,...,25.002400,15.5547,15.766400,20.753600,10.761600,10.5089,21.214300,30.90880,8.79188,13.20930
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,19.609300,18.158200,27.79620,86.77310,43.17470,63.108700,46.4904,20.757300,48.915800,58.970800,...,16.641200,27.7776,40.457400,43.506900,24.455700,50.7069,30.995300,55.96580,20.16570,18.87750
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.486023,0.497573,1.13765,1.12786,0.07823,0.114736,0.5554,0.260879,0.419212,0.706074,...,0.054276,0.0000,0.332898,0.835527,0.000000,0.0000,0.158432,0.56765,1.98550,5.37379


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
ARF5,92.642800,231.355000,197.86700,166.32700,239.04200,205.900000,105.2290,150.025000,152.680000,131.470000,...,134.466000,183.4860,90.798100,87.662600,230.199000,87.5706,110.064000,155.17500,81.53430,100.87900
M6PR,13.416100,37.246600,26.86280,32.23940,24.32130,27.363300,30.3292,30.646300,39.905200,34.483200,...,10.312700,0.0000,30.594200,32.774600,0.449073,3.1280,29.277800,42.09270,34.22870,44.67800
ESRRA,8.231030,15.566200,26.54580,27.97360,31.72550,30.049300,34.5154,29.175500,19.035300,21.015600,...,25.002400,15.5547,15.766400,20.753600,10.761600,10.5089,21.214300,30.90880,8.79188,13.20930
FKBP4,19.609300,18.158200,27.79620,86.77310,43.17470,63.108700,46.4904,20.757300,48.915800,58.970800,...,16.641200,27.7776,40.457400,43.506900,24.455700,50.7069,30.995300,55.96580,20.16570,18.87750
CYP26B1,0.486023,0.497573,1.13765,1.12786,0.07823,0.114736,0.5554,0.260879,0.419212,0.706074,...,0.054276,0.0000,0.332898,0.835527,0.000000,0.0000,0.158432,0.56765,1.98550,5.37379


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_luad_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
A1BG,0.036928,0.036928,0.036928,0.036928,0.036928,0.036928,0.036928,0.036928,0.036928,0.036928,...,0.036928,0.036928,0.036928,0.036928,0.248930,0.060128,0.036928,0.036928,0.036928,0.036928
A1CF,0.017037,0.047008,0.008880,0.255544,0.034242,0.019044,0.037996,0.025630,0.019733,0.035643,...,0.016972,0.100077,0.012967,0.055251,0.021455,0.016924,0.022493,0.017227,0.025194,0.008475
A2M,134.165000,455.107000,146.351000,95.827900,81.547000,162.367000,222.681000,130.718000,150.909000,146.615000,...,30.636900,216.082000,192.081000,61.516800,123.526000,122.010000,110.196000,39.874400,210.493000,1567.630000
A2ML1,0.038516,0.094817,0.155705,0.116177,0.118948,0.370544,0.145893,0.102505,0.208144,0.133007,...,0.099773,0.021208,0.034915,0.157368,0.203615,0.093850,0.100039,0.138855,0.110244,0.040664
A3GALT2,0.024241,0.070625,0.025939,0.042861,0.070625,0.070625,0.070625,0.044040,0.026117,0.071434,...,0.070625,0.220358,0.070625,0.020181,0.070625,0.069432,0.070625,0.019134,0.070625,0.060928


In [34]:
normalized_matrix.shape

(19021, 601)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
A1BG,-1.073994,-1.234411,-1.220093,-1.173298,-1.227191,-1.248640,-1.237957,-1.178610,-1.213372,-1.141817,...,-1.138734,-1.161951,-1.208793,-1.182040,-0.316970,-0.976461,-1.239614,-1.221987,-1.213032,-1.233526
A1CF,-1.409941,-1.129587,-1.839051,-0.333183,-1.259989,-1.536223,-1.225568,-1.337209,-1.485521,-1.157191,...,-1.476339,-0.728967,-1.663304,-1.007048,-1.381521,-1.527028,-1.454923,-1.553116,-1.379089,-1.872727
A2M,2.486295,2.856351,2.377952,2.240843,2.116866,2.394507,2.542375,2.370374,2.397992,2.457010,...,1.780160,2.605317,2.507341,2.039603,2.378711,2.330854,2.235201,1.811356,2.542855,3.394367
A2ML1,-1.055707,-0.824877,-0.595141,-0.675528,-0.719185,-0.247151,-0.641274,-0.735216,-0.462359,-0.585294,...,-0.707071,-1.402798,-1.233133,-0.552474,-0.404237,-0.783105,-0.806795,-0.646776,-0.738028,-1.191661
A3GALT2,-1.256796,-0.952804,-1.373488,-1.108591,-0.945584,-0.967033,-0.956350,-1.102112,-1.363800,-0.855265,...,-0.857128,-0.386173,-0.927186,-1.444450,-0.864089,-0.913984,-0.958007,-1.507541,-0.931425,-1.016060


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
A1BG,-1.167856,-1.328223,-1.196028,-1.213876,-1.173935,-1.192666,-1.186866,-1.199400,-1.252154,-1.229626,...,-1.006363,-1.466571,-1.124083,-1.103476,-0.463274,-0.931654,-1.177926,-1.069652,-1.195623,-1.198128
A1CF,-1.515047,-1.217318,-1.843141,-0.405559,-1.210037,-1.487494,-1.177518,-1.379503,-1.533700,-1.249400,...,-1.328223,-0.964616,-1.586297,-0.913648,-1.681384,-1.494606,-1.392521,-1.401264,-1.350354,-1.838247
A2M,2.553534,2.862674,2.339658,2.183814,2.226233,2.418809,2.564887,2.353949,2.365527,2.461306,...,1.868832,2.594325,2.552156,1.991557,2.357264,2.402579,2.313794,1.867762,2.597491,3.355894
A2ML1,-1.153400,-0.929182,-0.568569,-0.739507,-0.649033,-0.180332,-0.598069,-0.789483,-0.537745,-0.692784,...,-0.539835,-1.745869,-1.151437,-0.434451,-0.558001,-0.717173,-0.723742,-0.474052,-0.740875,-1.154554
A3GALT2,-1.356074,-1.055169,-1.348114,-1.151825,-0.891448,-0.901548,-0.913038,-1.132728,-1.402552,-0.958929,...,-0.699052,-0.594866,-0.827935,-1.361804,-1.065836,-0.871967,-0.884293,-1.348114,-0.927945,-0.980933


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-L9-A444-01,TCGA-MP-A4T9-01,TCGA-MP-A4TC-01,TCGA-MP-A4TA-01,TCGA-L4-A4E5-01,TCGA-MP-A4T8-01,TCGA-MP-A4T7-01,TCGA-MP-A4SW-01,TCGA-MP-A4SY-01,TCGA-MP-A4SV-01,...,TCGA-4B-A93V-01,TCGA-49-AAR0-01,TCGA-55-A48Y-01,TCGA-L9-A5IP-01,TCGA-L9-A50W-01,TCGA-55-A490-01,TCGA-NJ-A4YG-01,TCGA-MP-A4TE-01,TCGA-55-7816-01,TCGA-44-3398-11
A1BG,0.300425,-1.359497,0.008823,-0.175915,0.237500,0.043616,0.103654,-0.026082,-0.572130,-0.338940,...,1.971999,-2.791503,0.753501,0.966809,7.593383,2.745293,0.196193,1.316909,0.013015,-0.012916
A1CF,-0.152482,0.716819,-1.110442,3.086976,0.738079,-0.072033,0.833027,0.243275,-0.206945,0.623147,...,0.393003,1.454653,-0.360515,1.603470,-0.638149,-0.092798,0.205267,0.179740,0.328385,-1.096154
A2M,-0.085699,0.645724,-0.591729,-0.960457,-0.860094,-0.404459,-0.058839,-0.557917,-0.530524,-0.303912,...,-1.705703,0.010811,-0.088960,-1.415335,-0.550074,-0.442859,-0.652923,-1.708235,0.018301,1.812679
A2ML1,-1.497825,-0.682631,0.628457,0.006974,0.335915,2.039979,0.521206,-0.174725,0.740528,0.176849,...,0.732929,-3.651877,-1.490687,1.116074,0.666880,0.088176,0.064292,0.972098,0.002002,-1.502018
A3GALT2,-1.676462,-0.421098,-1.643254,-0.824343,0.261942,0.219804,0.171867,-0.744670,-1.870368,-0.019588,...,1.064611,1.499272,0.526914,-1.700366,-0.465600,0.343214,0.291791,-1.643254,0.109675,-0.111389


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19021, 577)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18508, 577)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [52]:
filename = path+'tcga_luad_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18508, 2)

# Save Gene List

In [56]:
filename = path+'tcga_luad_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-05-4244-01,70,MALE,M1,N2,T2,Stage IV,Lung Adenocarcinoma- Not Otherwise Specified (...,Primary Solid Tumor,Lung,NaN
TCGA-05-4249-01,67,MALE,M0,N0,T2,Stage IB,Lung Adenocarcinoma- Not Otherwise Specified (...,Primary Solid Tumor,Lung,NaN
TCGA-05-4250-01,79,FEMALE,M0,N1,T3,Stage IIIA,Lung Adenocarcinoma- Not Otherwise Specified (...,Primary Solid Tumor,Lung,NaN
TCGA-05-4382-01,68,MALE,M0,N0,T2,Stage IB,Lung Adenocarcinoma Mixed Subtype,Primary Solid Tumor,Lung,NaN
TCGA-05-4384-01,66,MALE,M0,N2,T2,Stage IIIA,Lung Adenocarcinoma Mixed Subtype,Primary Solid Tumor,Lung,NaN


In [60]:
attribute_list.shape

(577, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_luad_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4389-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,...,TCGA-NJ-A4YG-01,TCGA-NJ-A4YI-01,TCGA-NJ-A4YP-01,TCGA-NJ-A4YQ-01,TCGA-NJ-A55A-01,TCGA-NJ-A55O-01,TCGA-NJ-A55R-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0.178103,0.264819,0.594226,0.667036,0.646211,0.642752,0.275186,0.962018,-0.733178,0.778083,...,0.150370,0.968958,0.785012,0.188490,-0.875223,-0.071416,-0.989461,-0.767839,-0.365954,0.358402
A1CF,0.334327,-0.687961,0.819906,-0.092064,0.504214,0.948317,-0.909674,-0.258356,0.903189,-0.719128,...,0.143619,0.691524,0.244180,-0.351898,-0.157871,0.199102,0.927501,0.670705,0.608274,0.476512
A2M,-0.566901,0.296033,-0.210001,0.094966,0.667052,-0.750494,-0.383262,-0.927163,0.230153,-0.885589,...,-0.518376,0.032585,-0.449104,-0.258532,0.746822,0.088042,0.323755,-0.878662,0.271736,0.618490
A2ML1,-0.403200,0.411319,-0.933226,0.300368,-0.877779,0.837940,-0.936679,0.959438,-0.863940,0.775512,...,0.081996,0.296905,0.626341,0.019578,0.199814,0.754653,-0.382384,0.449446,0.661009,-0.320009
A3GALT2,-0.129899,0.365795,-0.812403,0.514929,-0.067465,0.598136,-0.556057,0.858361,-0.472891,-0.535281,...,0.310337,-0.206090,-0.978622,0.286093,-0.389738,0.140424,-0.022437,0.390100,0.948594,-0.857443


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_laud_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4389-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,...,TCGA-NJ-A4YG-01,TCGA-NJ-A4YI-01,TCGA-NJ-A4YP-01,TCGA-NJ-A4YQ-01,TCGA-NJ-A55A-01,TCGA-NJ-A55O-01,TCGA-NJ-A55R-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,-1,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_laud_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_laud_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_laud_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_laud_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_laud_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-05-4244, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4249, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4250, Age:79.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4382, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4384, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4389, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IA, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4390, Age:58.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4395, Age:76.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4396, Age:76.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-05-4397, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)",...,"(Donor:TCGA-NJ-A4YG, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A4YI, Age:87.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Lung Acinar Adenocarcinoma, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A4YP, Age:52.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A4YQ, Age:69.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IA, Histology:Lung Adenocarcinoma Mixed Subtype, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A55A, Age:76.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IB, Histology:Lung Acinar Adenocarcinoma, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A55O, Age:56.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIA, Histology:Mucinous (Colloid) Carcinoma, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A55R, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IA, Histology:Lung Solid Pattern Predominant Adenocarcinoma, Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-NJ-A7XG, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-O1-A52J, Age:74.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IA, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)","(Donor:TCGA-S2-AA1A, Age:68.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IA, Histology:Lung Bronchioloalveolar Carcinoma Mucinous, Tissue:Lung, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-05-4244, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Lung Adenocarcinoma- Not Otherwise Specified (NOS), Tissue:Lung, Tumor:nan, Gradenan)",1.000000,0.116251,0.116164,0.044817,0.077223,0.070381,0.001776,0.054139,0.024193,0.091860,...,0.053602,0.086879,0.038758,0.056962,0.075744,0.097993,0.082030,0.104218,0.117160,0.050940
"(Donor:TCGA-05-4249, 

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_laud_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.050600,-0.223034,-0.020990,0.024600,-0.047047,0.000062,0.043952,0.033852,-0.056517,...,0.091322,0.124839,-0.002690,0.002083,0.034457,0.065580,-0.063420,-0.098586,-0.121207,-0.025565
A1CF,-0.050600,1.000000,-0.079366,0.078808,0.087316,-0.146782,0.145562,0.066774,0.185960,0.133416,...,0.122535,0.092043,0.047071,0.156479,0.131589,0.002408,-0.098645,-0.174720,0.059655,0.030340
A2M,-0.223034,-0.079366,1.000000,-0.105544,-0.064057,0.031502,0.041357,-0.264650,-0.275701,0.325947,...,-0.538766,-0.625025,0.157208,0.043665,-0.080887,-0.412968,0.213665,0.438906,0.508554,-0.034615
A2ML1,-0.020990,0.078808,-0.105544,1.000000,0.012136,0.136076,0.013475,0.028856,0.060095,0.030004,...,0.158878,0.095526,0.052766,0.044581,0.148776,0.122440,-0.013401,-0.078032,0.065235,0.090018
A3GALT2,0.024600,0.087316,-0.064057,0.012136,1.000000,-0.076565,-0.044290,0.037913,-0.011439,-0.064985,...,0.079726,0.102709,-0.013141,0.036924,0.059225,0.080637,0.047061,-0.089007,0.000479,0.037038


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_luad_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_laud_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  577 Out of 577   

 The number of statisticaly relevent gene-attribute associations is: 533911
